Steps performed for data cleaning: 
1. @username removal 
2. urls removal 
3. spelling correction - speller & autocorrect 
4. hashtags collection
5. contractions expansion 
6. == as topic of wikipedia

In [1]:
# spelling correction library based on Norvig's implementation
!pip install pyspellchecker 

You should consider upgrading via the 'c:\users\lingl\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
# contractions expansion library
!pip install contractions 

You should consider upgrading via the 'c:\users\lingl\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd 
df = pd.read_csv("to_clean.csv")
df

,Text,oh_label,Label
0,`- This is not ``creative``. Those are the di...,0,0
1,` :: the term ``standard model`` is itself le...,0,0
2,"True or false, the situation as of March 200...",0,0
3,"Next, maybe you could work on being less cond...",0,0
4,This page will need disambiguation.,0,0
...,...,...,...
205173,228((( real!!!! by walt disney=,0,0
205174,Status-Online Im ZxkillergirlzX! I'm Zxkillerg...,0,0
205175,JR so cute EXO M Better I agree like yeah yeah...,0,0
205176,! !,0,0


In [4]:
df.Label.value_counts()

0    178629
1     25879
2       670
Name: Label, dtype: int64

In [5]:
# carry out stratified sampling of 10 records from each class first 
import numpy as np 
# sample = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(10))

In [6]:
df['user_dropped'] = ""

In [7]:
# drop all usernames
for index, row in df.iterrows():
    text_content = row['Text'].split(" ")
    user_dropped = []
    final = []
    for i in text_content:
        if "@" not in i:
            final.append(i)
    df.loc[index, 'user_dropped'] = " ".join(final)

In [8]:
df['url_dropped'] = ""

In [ ]:
# drop all urls
for index, row in df.iterrows():
    text_content = row['user_dropped'].split(" ")
    url_dropped = []
    final = []
    for i in text_content:
        if "http" not in i and "https" not in i and ".com" not in i and "www" not in i and ".co" not in i:
            final.append(i)
        df.loc[index, 'url_dropped'] = " ".join(final)

In [ ]:
df['spelling_corrected'] = ""

In [ ]:
# spelling correction 

from spellchecker import SpellChecker
spell = SpellChecker()

for index, row in df.iterrows():
    text_content = row['url_dropped'].split(" ")
    final = []
    for i in text_content:
        if i.startswith("#"):
            pass
        elif spell.unknown(i):
            i = spell.correction(i)
        final.append(i)
    df.loc[index, 'spelling_corrected'] = " ".join(final)

Findings: Spellchecker changes all slangs into formal words, which may be unfavourable for our analysis. As such, I performed the other types of cleaning on the data prior to any spelling corrections.

In [ ]:
df['hashtag_topics'] = ""

In [ ]:
# hastags as a topic 
for index, row in df.iterrows():
    text_content = row['url_dropped'].split(" ")
    final = []
    for i in text_content:
        if i.startswith("#"):
            final.append(i)
    df.loc[index, 'hashtag_topics'] = ",".join(final)

In [ ]:
df['expanded'] = ""

In [ ]:
# contractions expansion 
import contractions 
for index, row in df.iterrows():
    text_content = row['url_dropped'].split(" ")
    final = []
    for i in text_content:
        i = contractions.fix(i)
        final.append(i)
    df.loc[index, 'expanded'] = " ".join(final)

In [ ]:
# == as wikipedia topics 
for index, row in df.iterrows():
    text_content = row['url_dropped'] 
    if len(re.findall("==", text_content)) >= 2:
        occurences = [m.start() for m in re.finditer(r"==",text_content)]
        topic = text_content[occurences[0]+2:occurences[1]]
        topic = topic.strip()
#         print(topic)
        df.loc[index, 'wiki_topics'] = topic

In [ ]:
df.to_csv("clean2.csv")